In [ ]:
from matplotlib import pyplot as plt

# from lib_avena_vision.utils import VisionUtils
import cv2
import os

dataset_path = "/home/avena/avena_commons/resources/box_test_images"
name = "20251113_114330_515"
sufix = "_qr_color_frame.jpg"

color_image = cv2.imread(os.path.join(dataset_path, f"{name}{sufix}"))

fig, ax = plt.subplots(1, 1, figsize=(16, 9))
ax.imshow(cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB))

In [ ]:
from avena_commons.vision.detector.qr_detector import (
    qr_detector,
    create_qr_detection_visualization,
)

config = {
    "clahe": {"clip_limit": 4.0, "grid_size": 8},
    "middle_area": {"max_x": 1.44, "min_x": 0.56},
    "mode": "gray",
    "qr_size": 0.026,
}

frame = {"color": color_image, "depth": None}

camera_config = {
    "camera_params": [613.788, 613.756, 641.051, 402.142],
    "distortion_coefficients": [
        -0.0318751,
        0.0338157,
        -0.000153122,
        -0.000599062,
        -0.0119746,
    ],
}

detections, debug_data = qr_detector(
    frame=frame, config=config, camera_config=camera_config
)

print("Detections:", detections)
print("Debug data keys:", debug_data.keys())

visualization = create_qr_detection_visualization(
    color_image=frame["color"],
    detections=detections,
    timestamp=0.0,
    debug_dir="/home/avena/avena_commons/tests/temp/debug_frames",
)

fig, ax = plt.subplots(1, 1, figsize=(16, 9))
ax.imshow(cv2.cvtColor(visualization, cv2.COLOR_BGR2RGB))

In [ ]:
from avena_commons.vision.vision import calculate_pose_pnp
from avena_commons.vision.camera import create_camera_matrix
from avena_commons.vision.validation.transfor_to_base import transform_to_base

for detection in detections:
    qr_result = calculate_pose_pnp(
        corners=detection.corners,
        a=config["qr_size"] * 1000,
        b=config["qr_size"] * 1000,
        z=detection.z,
        camera_matrix=create_camera_matrix(camera_config["camera_params"]),
    )
    print("QR Pose:", qr_result)

    # qr_result = ( //Wygląda jakby tylko dla pozycji bez rotacji
    #     qr_result[0],  # x
    #     qr_result[1],  # y
    #     qr_result[2],  # z
    #     0.0,  # rx (zeroed)
    #     0.0,  # ry (zeroed)
    #     0.0,  # rz (zeroed)
    # )

    robot_position = [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ]  # Example robot position (x, y, z, rx, ry, rz)
    camera_tool_offset = [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ]  # Example camera tool offset (x, y, z, rx, ry, rz)

    position = transform_to_base(
        item=list(qr_result),
        current_tcp=robot_position,
        camera_tool_offset=self.__camera_config["camera_tool_offset"],
        is_rotation=False,
    )